In [16]:
import vtk
import xml.etree.ElementTree as ET
import os
import numpy as np

def ReadVTPFile(FileName):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader

def CutPlane(Volume,Origin,Norm):
    plane=vtk.vtkPlane()
    plane.SetOrigin(Origin)
    plane.SetNormal(Norm)
    Slice=vtk.vtkCutter()
    Slice.GenerateTrianglesOff()
    Slice.SetCutFunction(plane)
    Slice.SetInputData(Volume)
    Slice.Update()
    return Slice.GetOutput()

def ReadVTIFile(FileName):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(FileName)
    reader.Update()
    return reader.GetOutput()

def WriteVTPFile(FileName,Data):
    writer=vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(FileName)
    writer.SetInputData(Data)
    writer.Update()

def SphereClip(volume_image,center,radius):
    sphere = vtk.vtkSphere()
    sphere.SetCenter(center)
    sphere.SetRadius(radius)

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume_image)
    clipper.SetClipFunction(sphere)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def PlaneClip(volume,center, a):
    box = vtk.vtkBox()
    box.SetBounds(
        center[0] - a, center[0] + a,
        center[1] - a, center[1] + a,
        center[2] - a, center[2] + a
    )

    clipper = vtk.vtkClipDataSet()
    clipper.SetInputData(volume)
    clipper.SetClipFunction(box)
    clipper.InsideOutOn()
    clipper.GetOutputInformation(1)
    clipper.Update()

    return clipper.GetOutput()

def gradient_filter(vtk_image):
    gradient_filter = vtk.vtkImageGradient()
    gradient_filter.SetInputData(vtk_image)
    gradient_filter.SetDimensionality(3)
    gradient_filter.Update()

    return gradient_filter.GetOutput()
    
def define_borders(gradient_image):
    magnitude_filter = vtk.vtkImageMagnitude()
    magnitude_filter.SetInputData(gradient_image)
    magnitude_filter.Update()
    
    return magnitude_filter.GetOutput()

def ExtractSurface(UnstrcturedGrid):
    geometry_filter = vtk.vtkGeometryFilter()
    geometry_filter.SetInputData(UnstrcturedGrid)
    geometry_filter.Update()

    return geometry_filter.GetOutput()

def WriteVTUFile(FileName,Data):
	writer=vtk.vtkXMLUnstructuredGridWriter()
	writer.SetFileName(FileName)
	writer.SetInputData(Data)
	writer.Update()

def clip_polydata_with_plane(polydata, origin, normal, inside_out=False):
    plane = vtk.vtkPlane()
    plane.SetOrigin(origin)
    plane.SetNormal(normal)

    clipper = vtk.vtkClipPolyData()
    clipper.SetInputData(polydata)
    clipper.SetClipFunction(plane)

    if inside_out:
        clipper.InsideOutOn()
    else:
        clipper.InsideOutOff()
    
    clipper.Update()

    return clipper.GetOutput()

In [17]:
pathline_file = "/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Paths/L_LAD_0.pth"
ImagePath = "/Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Images/ct.vti"

In [18]:
with open(pathline_file, "r") as path:
    #path.readlines()
    tree = ET.parse(path)
root = tree.getroot()

direction_points = []
for direction_point in root.findall(".//path_point/tangent"):
    x = float(direction_point.attrib['x'])
    y = float(direction_point.attrib['y'])
    z = float(direction_point.attrib['z'])
    direction_points.append((x,y,z))

path_points = []
for path_point in root.findall(".//path_point/pos"):
    x = float(path_point.attrib['x'])
    y = float(path_point.attrib['y'])
    z = float(path_point.attrib['z'])
    path_points.append((x,y,z))

path_normals = []
for normal in root.findall(".//path_point/rotation"):
    x = float(normal.attrib['x'])
    y = float(normal.attrib['y'])
    z = float(normal.attrib['z'])
    path_normals.append((x,y,z))

NPoints = len(path_points)
print("the number of points in the centerline is:", NPoints)

binormal = []
for i in range(NPoints):
    binormal_ = np.cross(np.array(direction_points[i]),np.array(path_normals[i]))
    binormal_ /= np.linalg.norm(binormal_)
    binormal.append(binormal_)
    

the number of points in the centerline is: 101


In [19]:
Volume = ReadVTIFile(ImagePath)
Volume_Gradient = define_borders(gradient_filter(Volume))

In [ ]:
#SKIP

a = 10
cs_loc = 100
CrossSection = CutPlane(SphereClip(Volume,path_points[cs_loc],2*a),path_points[cs_loc],direction_points[cs_loc])
CrossSectionGrd = CutPlane(SphereClip(Volume_Gradient,path_points[cs_loc],2*a),path_points[cs_loc],direction_points[cs_loc])

In [ ]:
#SKIP

normal = np.array(path_normals[1])
Bnormal = np.array(binormal[1])
#print(normal[0], normal[1], normal[2])
Rays = []
for _,array in enumerate([normal, -1*normal, Bnormal, -1*Bnormal]):
    print(array)
    print(array[0], array[1], array[2])
    #ray_ = np.array(CenterPoint[0] + 100 * array[0], CenterPoint[1] + 100 * array[1], CenterPoint[2] + 100 * array[2])
    #Rays.append(ray_)


[0.         0.27219775 0.96224133]
0.0 0.272197749777385 0.962241334082115
[-0.         -0.27219775 -0.96224133]
-0.0 -0.272197749777385 -0.962241334082115
[-0.61499203 -0.75875936  0.214637  ]
-0.6149920316840819 -0.758759356376682 0.21463699605596534
[ 0.61499203  0.75875936 -0.214637  ]
0.6149920316840819 0.758759356376682 -0.21463699605596534


In [20]:

def EstimateVesselRadius(CrossSectionGradient, CenterPoint, normal, Bnormal):
    
    normal = np.array(normal)
    Bnormal = np.array(Bnormal)

    Rays = []
    for arr in [normal, Bnormal]:
        ray_ = np.array([CenterPoint[0] + 100 * arr[0], CenterPoint[1] + 100 * arr[1], CenterPoint[2] + 100 * arr[2]])
        Rays.append(ray_)

    #Radius = []
    #for ray in Rays:
    line = vtk.vtkLineSource()
    line.SetPoint1(CenterPoint)
    line.SetPoint2(Rays[0])
    line.SetResolution(100)

    probe = vtk.vtkProbeFilter()
    probe.SetInputConnection(line.GetOutputPort())
    probe.SetSourceData(CrossSectionGradient)
    probe.Update()

    sampled_data = probe.GetOutput().GetPointData().GetScalars()
    gradients = np.array([sampled_data.GetTuple1(i) for i in range(sampled_data.GetNumberOfTuples())])

    boundary_idx = np.argmax(gradients)

    boundary_point = probe.GetOutput().GetPoint(boundary_idx)
    radius = np.sqrt((boundary_point[0] - CenterPoint[0])**2 + (boundary_point[1] - CenterPoint[1])**2 + (boundary_point[2] - CenterPoint[2])**2)
    #Radius.append(radius)

    return radius#np.mean(np.array(Radius))


In [ ]:
#SKIP

OutputFolder = os.path.dirname(pathline_file)
VesselName = os.path.splitext(os.path.basename(pathline_file))[0]
OutputFolder = os.path.join(OutputFolder,VesselName)

for cs_loc in range(NPoints):
    CrossSectionGrd = CutPlane(SphereClip(Volume_Gradient,path_points[cs_loc],2*a),path_points[cs_loc],direction_points[cs_loc])
    radius = EstimateVesselRadius(CrossSectionGrd, path_points[cs_loc], path_normals[cs_loc], binormal[cs_loc])
    CS_Grd_FileName = f"{VesselName}_CrossSection_{cs_loc}_Grd.vtp"
    WriteVTPFile(os.path.join(OutputFolder,CS_Grd_FileName), CrossSectionGrd)


KeyboardInterrupt: 

In [ ]:
#SKIP

cs_loc = 100
CrossSectionGrd = CutPlane(SphereClip(Volume_Gradient,path_points[cs_loc],2*a),path_points[cs_loc],direction_points[cs_loc])
radius = EstimateVesselRadius(CrossSectionGrd, path_points[cs_loc], path_normals[cs_loc], binormal[cs_loc])
print(radius)

0.9999932359167059


In [ ]:
#SKIP

sphere = vtk.vtkSphereSource()
sphere.SetCenter(path_points[cs_loc])
sphere.SetRadius(radius)
sphere.Update()


In [21]:
pathline_vtp_file = f"{os.path.splitext(pathline_file)[0]}.vtp"
CenterLineVTP = ReadVTPFile(pathline_vtp_file)

In [22]:
from scipy.interpolate import interp1d

radius_scalars = vtk.vtkDoubleArray()
radius_scalars.SetNumberOfComponents(1)
radius_scalars.SetName("Radius")

a = 10
Radius = []
for i in range(NPoints):
    CrossSectionGrd = CutPlane(SphereClip(Volume_Gradient,path_points[i],2*a),path_points[i],direction_points[i])
    radius = EstimateVesselRadius(CrossSectionGrd, path_points[i], path_normals[i], binormal[i])
    Radius.append(radius)
    #radius_scalars.InsertNextValue(radius)



In [25]:
def CleanRadius(Radius, start_point = 4.0, end_point = 0.9):
    Radius = np.array(Radius)
    NPoints = len(Radius)

    expected_radii = np.linspace(start_point,end_point, NPoints)
    min_allowed = 0.25*expected_radii
    max_allowed = 1.1*expected_radii

    outliers = (Radius < min_allowed) | (Radius > max_allowed)

    valid_indices = np.where(~outliers)[0]
    valid_radii = Radius[valid_indices]

    interp_func = interp1d(valid_indices, valid_radii, kind='linear', fill_value='extrapolate')

    cleaned_radius = Radius.copy()
    cleaned_radius[outliers] = interp_func(np.where(outliers)[0])

    return cleaned_radius

CleanedRadius = CleanRadius(Radius)


In [26]:
def MovingAverageFilter(input_array, window_size = 5):
    return np.convolve(input_array, np.ones(window_size)/window_size, mode='same')

SmoothedRadius = MovingAverageFilter(CleanedRadius,5)

from scipy.ndimage import gaussian_filter1d

SmoothedRadius = gaussian_filter1d(CleanedRadius, sigma = 1)


for i in range(NPoints):
    print(Radius[i], CleanedRadius[i], SmoothedRadius[i])
    radius_scalars.InsertNextValue(SmoothedRadius[i])

4.000001682849798 4.000001682849798 4.00000205342398
4.000003056821445 4.000003056821445 4.000001933999865
4.00000138344407 4.00000138344407 4.000000745050004
3.9999977341123523 3.9999977341123523 3.999999428317947
3.9999975798709473 3.9999975798709473 3.9998665824709847
4.0000076017260255 4.0000076017260255 3.9955704549281137
4.000000000890276 4.000000000890276 3.94587666171505
3.9999997973206383 3.9999997973206383 3.7534632648775634
2.999998906096958 2.999998906096958 3.542500817296525
4.00000645822972 4.00000645822972 3.4575025254225524
3.0000000650732086 3.0000000650732086 3.2465384898415275
2.999998071905698 2.999998071905698 3.053992154148707
3.000005055481283 3.000005055481283 2.999868806401921
3.000003708907903 3.000003708907903 2.9415357218998346
3.000002891850904 3.000002891850904 2.6979077351441845
2.0000001816151545 2.0000001816151545 2.27944456955871
2.000002548814144 2.000002548814144 1.9376095033611747
4.999995300123048 1.6666690757763072 1.6608470438463718
9.00000069642

In [27]:
CenterLine = CenterLineVTP.GetOutput()
CenterLine.GetPointData().SetScalars(radius_scalars)

0

In [28]:
def Line(centerline):
    tube_filter = vtk.vtkTubeFilter()
    tube_filter.SetInputData(centerline)
    #tube_filter.SetRadius(Radius)  # Adjust this value to change thickness
    tube_filter.SetNumberOfSides(50)  # Higher = smoother tube
    #tube_filter.CappingOn()  # Close tube ends
    tube_filter.SetVaryRadiusToVaryRadiusByAbsoluteScalar()
    tube_filter.Update()
    return tube_filter.GetOutput()

VesselSegmentation = Line(CenterLine)

In [29]:
OutputFolder = os.path.dirname(pathline_file)
VesselName = os.path.splitext(os.path.basename(pathline_file))[0]
OutputFolder = os.path.join(OutputFolder,VesselName)

os.system(f"mkdir {OutputFolder}")

mkdir: /Users/ana/Documents/AnahitaSeresti/Tesselation/KoenTesselation_SU03A/SimVascular_Edgard/Paths/L_LAD_0: File exists


256

In [30]:
#CS_Grd_FileName = f"{VesselName}_CrossSection_{cs_loc}_Grd.vtp"
#WriteVTPFile(os.path.join(OutputFolder,CS_Grd_FileName), CrossSectionGrd)

OutputFileName = f"{VesselName}_Segementation.vtp"
WriteVTPFile(os.path.join(OutputFolder,OutputFileName), VesselSegmentation)